In [1]:
import os
import sys
import pretty_errors
import PyPDF2
from unstructured.partition.pdf import partition_pdf

/workspaces/software_effort_estimation_rag/venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


The PDF contains several pages with irrelevant or uninformative content. To optimize processing, we will first identify and select only the relevant pages before extracting elements from them.

In [ ]:
irrelevant_content_pages = list(range(0, 9)) + list(range(51, 54))

with open('../see_report.pdf', 'rb') as file:
    reader = PyPDF2.PdfReader(file)
    total_pages = len(reader.pages)
    writer = PyPDF2.PdfWriter()
 
    remaining_pages = filter(lambda x: x not in irrelevant_content_pages, range(total_pages))   

    for page_num in remaining_pages:
        writer.add_page(reader.pages[page_num])
        

In [3]:
raw_pdf_elements = partition_pdf('../see_report.pdf', 
                    strategy='hi_res', 
                    pages=list(range(9, total_pages+1)), 
                    extract_images_in_pdf=True,
                    infer_table_structure=True,
                    chunking_strategy="by_title",)

Cannot set gray non-stroke color because /'P28' is an invalid float value


Cannot set gray non-stroke color because /'P28' is an invalid float value
The `max_size` parameter is deprecated and will be removed in v4.26. Please specify in `size['longest_edge'] instead`.


In [5]:
from tqdm import tqdm

tables, texts, images = [], [], []

for element in tqdm(raw_pdf_elements):
    if "unstructured.documents.elements.Table" in str(type(element)):
        tables.append(str(element))
    elif "unstructured.documents.elements.CompositeElement" in str(type(element)):
        texts.append(str(element))
    elif "unstructured.documents.elements.Image" in str(type(element)):
        images.append(element)

100%|██████████| 262/262 [00:00<00:00, 405351.40it/s]
